In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!cp -r '/content/gdrive/MyDrive/Colab Notebooks/severstal-steel-defect-detection.zip' './'
!unzip './severstal-steel-defect-detection.zip'

串流輸出內容已截斷至最後 5000 行。
  inflating: train_images/99f75320d.jpg  
  inflating: train_images/99f9d2375.jpg  
  inflating: train_images/99fd3c6f5.jpg  
  inflating: train_images/9a016fe15.jpg  
  inflating: train_images/9a064450d.jpg  
  inflating: train_images/9a08c2783.jpg  
  inflating: train_images/9a18e4457.jpg  
  inflating: train_images/9a1f7c238.jpg  
  inflating: train_images/9a2523ce9.jpg  
  inflating: train_images/9a3e774ff.jpg  
  inflating: train_images/9a5e9e77c.jpg  
  inflating: train_images/9a5f7a855.jpg  
  inflating: train_images/9a62177bb.jpg  
  inflating: train_images/9a67575cf.jpg  
  inflating: train_images/9a6ac5406.jpg  
  inflating: train_images/9a70057c0.jpg  
  inflating: train_images/9a72fd89e.jpg  
  inflating: train_images/9a75974ba.jpg  
  inflating: train_images/9a762b892.jpg  
  inflating: train_images/9a7b2f3af.jpg  
  inflating: train_images/9a7b427b4.jpg  
  inflating: train_images/9a81a8056.jpg  
  inflating: train_images/9a83c23d1.jpg  
  inflating: 

In [3]:
!pip install segmentation_models

     |████████████████████████████████| 51kB 7.2MB/s 


In [4]:
!pip install q tensorflow==2.1
!pip install q keras==2.3.1

     |████████████████████████████████| 421.8MB 19kB/s 
     |████████████████████████████████| 3.9MB 48.8MB/s 
     |████████████████████████████████| 450kB 47.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=f8a7798cb91bfce29e6f9d8ee8a4b01229efe9e1605570665addad21ba53e2d8
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: 

In [5]:
# Import libraries
import warnings
warnings.filterwarnings("ignore")

from time import time
import pandas as pd
import numpy as np
import os 
import cv2
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import segmentation_models
print(segmentation_models.__version__)

from sklearn.metrics import classification_report, multilabel_confusion_matrix

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.layers import GlobalAveragePooling2D, Dense, Conv2D, BatchNormalization, Dropout
from keras.models import Model, load_model
import tensorflow as tf
from tensorflow.python.keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import seaborn
import segmentation_models as sm
from segmentation_models import Unet
from segmentation_models import get_preprocessing

import albumentations as A
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import Sequence

Using TensorFlow backend.


Segmentation Models: using `keras` framework.
1.0.1


In [6]:
tf.config.optimizer.set_jit(True)


Utility functions, metrics and DataGenerator

In [7]:
# We need a function to convert EncodedPixels into mask
# https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(mask_rle, shape=(1600,256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

# Below metrics are useful for loading saved models

# https://www.kaggle.com/xhlulu/severstal-simple-keras-u-net-boilerplate
# COMPETITION METRIC 
def dice_coef(y_true, y_pred, smooth=K.epsilon()):
    '''
    This function returns dice coefficient of similarity between y_true and y_pred
    Dice coefficient is also referred to as F1_score, but we will use this name for image segmentation models
    For example, 
    let an instance on y_true and y_pred be [[1,1],[0,1]] and [[1,0],[0,1]]
    this metric first converts the above into [1,1,0,1] abd [1,0,0,1],
    then intersection is calculated as 1*1 + 1*0 + 0*1 + 1*1 = 2 and sum(y_true)+sum(y_pred)= 3+2 = 5
    this returns the value (2.* 2 + 10e-7)/(3 + 2 + 10e-7) ~ 0.8    
    '''
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Custom metrics, https://stackoverflow.com/questions/59196793/why-are-my-metrics-of-my-cnn-not-changing-with-each-epoch
def recall_m(y_true, y_pred):
    '''
    This function returns recall_score between y_true and y_pred
    This function is ported as a metric to the Neural Network Models
    Keras backend is used to take care of batch type training, the metric takes in a batch of y_pred and corresponding y_pred 
    as input and returns recall score of the batch
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) # calculates number of true positives
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))      # calculates number of actual positives
    recall = true_positives / (possible_positives + K.epsilon())   # K.epsilon takes care of non-zero divisions
    return recall

def precision_m(y_true, y_pred):
    '''
    This function returns precison_score between y_true and y_pred
    This function is ported as a metric to the Neural Network Models
    Keras backend is used to take care of batch type training, the metric takes in a batch of y_pred and corresponding y_pred 
    as input and returns prediction score of the batch
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))  # calculates number of true positives
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))      # calculates number of predicted positives   
    precision = true_positives /(predicted_positives + K.epsilon()) # K.epsilon takes care of non-zero divisions
    return precision
    
def f1_score_m(y_true, y_pred):
    '''
    This function returns f1_score between y_true and y_pred
    This 
    This function is ported as a metric to the Neural Network Models
    Keras backend is used to take care of batch type training, the metric takes in a batch of y_pred and corresponding y_pred 
    as input and returns f1 score of the batch
    '''
    precision = precision_m(y_true, y_pred)  # calls precision metric and takes the score of precision of the batch
    recall = recall_m(y_true, y_pred)        # calls recall metric and takes the score of precision of the batch
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def area(i):
    '''
    Input: EncodedPixels (str)
    Output: number of pixels having the defect
    '''
    return sum([int(k) for k in i.split(' ')[1::2]])

class test_DataGenerator_3(keras.utils.Sequence):
    def __init__(self, df, batch_size = 1, image_path = './test_images/',
                 preprocess=None, info={}):
        super().__init__()
        self.df = df
        self.batch_size = batch_size
        self.preprocess = preprocess
        self.info = info
        self.data_path = image_path
        self.on_epoch_end()
    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
    def __getitem__(self, index): 
        '''
        The DataGenerator takes ImageIds of batch size 1 and returns Image array to the model.
        With the help of ImageIds the DataGenerator locates the Image file in the path, the image is read and resized from
        256x1600 to 256x800.
        '''
        X = np.empty((self.batch_size,256,800,3),dtype=np.float32)
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['ImageId'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            X[i,] = Image.open(self.data_path + f).resize((800,256))      
        if self.preprocess!=None: X = self.preprocess(X)
        return X


In [8]:
class AugmentDataGenerator(Sequence):
    def __init__(self, datagen, augment=None):
        self.datagen = datagen
        if augment is None:
            self.augment = A.Compose([])
        else:
            self.augment = augment

    def __len__(self):
        return len(self.datagen)

    def __getitem__(self, x):
        images, *rest = self.datagen[x]
        augmented = []
        for image in images:
            image = self.augment(image=image)['image']
            augmented.append(image)
        return (np.array(augmented), *rest)

Loading Data and saved Models

In [9]:
#! 7z e './severstal-steel-defect-detection.zip' -oraw_data   train_images/*.jpg  # https://stackoverflow.com/questions/31984387/command-line-for-7z-to-extract-specific-files-from-specific-folders-inside-an-ar
train_image_names = os.listdir('./train_images/')

#! 7z e './severstal-steel-defect-detection.zip' -oraw_data   test_images/*.jpg
test_image_names = os.listdir('./test_images/')

#! 7z e './severstal-steel-defect-detection.zip' -oraw_data   train.csv
trainLabels = pd.read_csv('./train.csv')

In [10]:
dependencies = {
        'recall_m':recall_m,
        'precision_m':precision_m,
        'dice_coef':dice_coef,
        'f1_score_m':f1_score_m,
        'dice_loss':sm.losses.dice_loss
    }

Load Models

In [11]:
print('Load binary model in ...')
model_binary = load_model('/content/gdrive/MyDrive/Colab Notebooks/final_project/binary_model_1204.hdf5', custom_objects=dependencies)
print('Load multi class model in ...')
model_multi = load_model('/content/gdrive/MyDrive/Colab Notebooks/final_project/multi_class_model_1229.h5', custom_objects=dependencies) #0105
print('Load segmentation models in ...')
model_segment_1 = load_model('/content/gdrive/MyDrive/Colab Notebooks/segment_model_defect1_0110_2.h5', custom_objects=dependencies)
model_segment_2 = load_model('/content/gdrive/MyDrive/Colab Notebooks/segment_model_defect2_0110_2.h5', custom_objects=dependencies) #0105
model_segment_3 = load_model('/content/gdrive/MyDrive/Colab Notebooks/segment_model_defect3_0110_2.h5', custom_objects=dependencies)
model_segment_4 = load_model('/content/gdrive/MyDrive/Colab Notebooks/segment_model_defect4_0110_2.h5', custom_objects=dependencies)

Load binary model in ...
Load multi class model in ...
Load segmentation models in ...


Predictor functions

In [12]:
def pred_classification(X):
    '''
    Input: ImageIds in form of a dataframe
    Return: Predictions of classification models
    '''
    X = X.reset_index().drop('index',axis=1)
    data_generator = ImageDataGenerator(rescale=1./255).flow_from_dataframe(dataframe=X, directory='./test_images/',
                                                                            x_col="ImageId", class_mode = None, 
                                                                            target_size=(256,512), batch_size=1, shuffle=False)

    data_preds_binary = model_binary.predict_generator(data_generator,verbose=0)
    data_preds_multi_label = model_multi.predict_generator(data_generator,verbose=0)
    data_classification = pd.DataFrame(data_preds_multi_label, columns = ['defect_1','defect_2','defect_3','defect_4'])
    data_classification['hasDefect'] = data_preds_binary
    data_classification['ImageId'] = X['ImageId']
    return data_classification[['ImageId','hasDefect','defect_1','defect_2','defect_3','defect_4']]

In [13]:
def pred_segmentation(X):
    '''
    Input: ImageIds in form of a dataframe
    Return: Predictions of segmentation models
    '''
    X = X.reset_index().drop('index',axis=1)
    preprocess = get_preprocessing('efficientnetb2')
    tmp=[]
    loop_num = 50
    for j in range((len(X)//loop_num)+1):
        test_dataf = X[loop_num*j:loop_num*j+loop_num]
        test_batches =  test_DataGenerator_3(test_dataf,preprocess=preprocess)
        test_preds_1 = model_segment_1.predict_generator(test_batches,verbose=0)
        test_preds_2 = model_segment_2.predict_generator(test_batches,verbose=0)
        test_preds_3 = model_segment_3.predict_generator(test_batches,verbose=0)
        test_preds_4 = model_segment_4.predict_generator(test_batches,verbose=0)
        for i in range(len(test_preds_1)):
            ep1 = mask2rle(np.array((Image.fromarray((test_preds_1[i][:,:,0])>=0.5)).resize((1600,256))).astype(int))
            ep2 = mask2rle(np.array((Image.fromarray((test_preds_2[i][:,:,0])>=0.5)).resize((1600,256))).astype(int))
            ep3 = mask2rle(np.array((Image.fromarray((test_preds_3[i][:,:,0])>=0.5)).resize((1600,256))).astype(int))
            ep4 = mask2rle(np.array((Image.fromarray((test_preds_4[i][:,:,0])>=0.5)).resize((1600,256))).astype(int))
            tmp.append([test_dataf.ImageId.iloc[i],ep1,ep2,ep3,ep4])

    return pd.DataFrame(tmp,columns=['ImageId','EncodedPixels_1','EncodedPixels_2','EncodedPixels_3','EncodedPixels_4'])

In [18]:
def pred_combined(X):
    '''
    Input: ImageId (dataframe)
    Return: Comdined dataframe of output of pred_classification function and pred_segmentation function
    '''
    X = X.reset_index().drop('index',axis=1)
    return pred_classification(X).merge(pred_segmentation(X),on=['ImageId'])
    
def steel_prediction(X):
    '''
    Function-1:
    Input: ImageId(dataframe)
    Process: Calls pred_combined which calls pred_classification and pred_segmentation
            Applies thresholds -> area and classification probability
    Return: DataFrame (columns = ImageId_ClassId,EncodedPixels)
    
    '''
    X = X.reset_index().drop('index',axis=1)
    p = pred_combined(X)
    tmp = []
    for i in range(len(p)):        
        j, b, m1, m2, m3, m4, ep1, ep2, ep3, ep4 = p.iloc[i]
        # randomly selected classification threshold values to get high recall 
        # for no defect binary classifier and high precision for multi-label classifier 
        # while not compromising much on other metrics
        
        # area thresholds are determined from EDA performed only on train dataset

        # ------------- threshold can modified ------------------ #

        if area(ep1)>=500 and area(ep1)<=15500 and b>=0.95 and m1>=0.5: 
            tmp.append([j+'_1',ep1])
        else:
            tmp.append([j+'_1',''])

        if area(ep2)>=700 and area(ep2)<=10000 and b>=0.85 and m2>=0.5:
            tmp.append([j+'_2',ep2])
        else:
            tmp.append([j+'_2',''])

        if area(ep3)>=1100 and area(ep3)<=160000 and b>=0.85 and m3>=0.75:
            tmp.append([j+'_3',ep3])
        else:
            tmp.append([j+'_3',''])
            
        if area(ep4)>=2800 and area(ep4)<=127000 and b>=0.85 and m4>=0.5:
            tmp.append([j+'_4',ep4])
        else:
            tmp.append([j+'_4',''])
    return pd.DataFrame(tmp, columns = ['ImageId_ClassId','EncodedPixels'])


steel_prediction() is final functions defined for generating predictions

# ***MAKE PREDICT***

Determining time taken by the model for generating predictions on a new unseen image:

In [16]:
raw_test = pd.DataFrame(test_image_names,columns=['ImageId'])

Generating Predictions on raw test dataset for submission on Kaggle

In [19]:
c2 = steel_prediction(raw_test)
c2.to_csv('/content/gdrive/MyDrive/Colab Notebooks/severstal_final_test_preds_0111_3.csv',index=False)
c2.shape


Found 5506 validated image filenames.


(22024, 2)

---

# ***Evaluation(unfinish)***